In [ ]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

import os


## Kernel Density Estimate Plots

In [ ]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df,label_name):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df[label_name].corr(df[var_name])
            
    plt.figure(figsize = (12, 6))
    for value in df[label_name].unique():
        # Plot the distribution for target == 0 and target == 1
        sns.kdeplot(df.ix[df[label_name] == value, var_name], label = 'Label : '+ str(value))

    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    for value in df[label_name].unique():
        # Calculate medians for repaid vs not repaid
        avg_label_val = df.ix[df[label_name] == value, var_name].median()
        print('Median value for {%s} =     %0.4f' %(value ,avg_label_val))
    # print out the correlation
    print('The correlation between %s and the Target variable is %0.4f' % (var_name, corr))
    